In [0]:
tables_to_deprecate = spark.sql(
    "select collect_set(full_table_name) from quality.default.inventory where to_be_deprecated = True"
).first()[0]

In [0]:
tables_to_deprecate

https://docs.databricks.com/aws/en/data-governance/unity-catalog/certify-deprecate-data

In [0]:
for  tab in tables_to_deprecate:
  spark.sql(f"SET TAG ON TABLE {tab} `system.certification_status` = `deprecated`")

In [0]:
tables_to_delete = spark.sql("""
select collect_set(request_params.securable_full_name) from quality.default.audit_hist 
where action_name = 'CreateSecurableTagAssignment' 
and response.status_code = 200 
and event_date <= current_date - INTERVAL 30 DAYS
and request_params.tag_value = 'deprecated'
and request_params.tag_key = 'system.certification_status' 
          """).first()[0]


In [0]:
# for tab in tables_to_delete:
#   spark.sql(f"DROP TABLE IF EXISTS {tab}")